## Imports

In [3]:
import pymysql.cursors
import configparser
import sqlalchemy
import os

In [70]:
import pandas as pd
import sklearn
from sklearn.pipeline import Pipeline, make_pipeline, TransformerMixin
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn import preprocessing, ensemble, linear_model

In [16]:
print(sklearn.__version__)
print(pd.__version__)

0.18.1
0.19.2


## Functions

In [7]:
def connect_to_database(credentials_path, group):
    reader = configparser.RawConfigParser()
    reader.read(os.path.join(credentials_path, '.my.cnf'))
    connection_string = 'mysql+pymysql://{user}:{password}@{host}:{port}/{dbname}'.format(
        user = reader.get(group, 'user'),
        password = reader.get(group, 'password'),
        host = reader.get(group, 'host'),
        port = reader.get(group, 'port'),
        dbname = reader.get(group, 'database')
    )
    engine = sqlalchemy.create_engine(connection_string)
    return engine

In [8]:
def convert_categorical(data):
    binary_cols = set(col for col in data if (data[col].nunique() == 2 and not col.endswith('counts')))
    string_cols = set(col for col in data if data[col].dtype == 'O')
    categoricals = binary_cols.union(string_cols)
    categoricals.add('appl_year')
    data_categorical = data.astype({col: 'category' for col in categoricals})
    return data_categorical

## Connect to database

In [9]:
group = 'edu_db_owner'
credentials_path = '/Volumes/IIME/EDS/data/admissions/db_credentials'

engine = connect_to_database(credentials_path, group)

In [10]:
tbl_name = 'grades_gpa_mcat_other_features'
get_table = 'select * from deidentified$model_data${}'.format(tbl_name)

model_matrix = pd.read_sql_query(get_table, engine)
model_matrix = model_matrix.set_index('study_id')

In [11]:
model_matrix.shape

(1218, 54)

## Prepare data for modeling

In [54]:
model_matrix = model_matrix.astype({'appl_year': int})
model_matrix = convert_categorical(model_matrix)

In [56]:
model_matrix.dtypes

appl_year                        category
outcome                          category
bcpm_gpa_cumulative               float64
bcpm_hours_cumulative             float64
total_gpa_cumulative              float64
bcpm_gpa_postbac                  float64
bcpm_hours_postbac                float64
total_gpa_postbac                 float64
total_gpa_soph_higher            category
total_gpa_juni_higher            category
total_gpa_seni_higher            category
bcmp_gpa_soph_higher             category
bcmp_gpa_juni_higher             category
bcmp_gpa_seni_higher             category
did_postbac                      category
attended_grad_school             category
bcpm_a_counts                     float64
bcpm_b_counts                     float64
bcpm_c_counts                     float64
bcpm_d_counts                     float64
bcpm_f_counts                     float64
math_a_counts                     float64
math_b_counts                     float64
math_c_counts                     

In [57]:
X, y = model_matrix.drop('outcome', axis = 1), model_matrix.outcome

In [58]:
X_train, X_test, y_train, y_test = train_test_split(X, y, 
                                    test_size=0.20, random_state=1100)

In [103]:
y_train, y_test = y_train.astype(int), y_test.astype(int)

In [84]:
class DummyEncoder(sklearn.base.BaseEstimator, TransformerMixin):

    def __init__(self, columns=None):
        self.columns = columns

    def transform(self, X, y=None, **kwargs):
        return pd.get_dummies(X, columns = self.columns, 
            drop_first = True, dummy_na = True)

    def fit(self, X, y=None, **kwargs):
        return self

In [85]:
enc = DummyEncoder().fit(X_train)
X_train_transformed = enc.transform(X_train)
X_test_transformed = enc.transform(X_test)
X_train_transformed.columns == X_test_transformed.columns

array([ True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True,  True,  True,  True,
        True,  True,  True,  True,  True,  True], dtype=bool)

In [105]:
param_grid = dict(
    imputer__strategy= ['mean', 'median'],
    randomforestclassifier__n_estimators= [10, 50], # [50, 100, 200, 500, 1000],
    randomforestclassifier__max_depth= [1, 5, 10], #[1, 5, 10, 20, 50],
    randomforestclassifier__max_features= ['sqrt', 'log2'],
    randomforestclassifier__min_samples_split= [2, 5, 10])

In [106]:
pipeline = make_pipeline(DummyEncoder(), preprocessing.Imputer(), ensemble.RandomForestClassifier())

In [107]:
grid_search = GridSearchCV(pipeline, param_grid = param_grid)

In [109]:
grid_search.fit(X_train, y_train)

GridSearchCV(cv=None, error_score='raise',
       estimator=Pipeline(steps=[('dummyencoder', DummyEncoder(columns=None)), ('imputer', Imputer(axis=0, copy=True, missing_values='NaN', strategy='mean', verbose=0)), ('randomforestclassifier', RandomForestClassifier(bootstrap=True, class_weight=None, criterion='gini',
            max_depth=None, max_features='auto', max_...imators=10, n_jobs=1, oob_score=False, random_state=None,
            verbose=0, warm_start=False))]),
       fit_params={}, iid=True, n_jobs=1,
       param_grid={'imputer__strategy': ['mean', 'median'], 'randomforestclassifier__n_estimators': [10, 50], 'randomforestclassifier__max_depth': [1, 5, 10], 'randomforestclassifier__max_features': ['sqrt', 'log2'], 'randomforestclassifier__min_samples_split': [2, 5, 10]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)